In [0]:
l = range(5)

In [0]:
l

Out[2]: range(0, 5)

In [0]:
rdd = sc.parallelize(l)

In [0]:
rdd.collect()

Out[50]: [0, 1, 2, 3, 4]

In [0]:
def sqr(e):
    return e**2

rdd.map(sqr).collect()

Out[51]: [0, 1, 4, 9, 16]

#### display the sum of a rdd using reduce

In [0]:
rdd.reduce( lambda s,e :print(s,e) )

0 1
None 2
None 3
None 4


#### Calculate the sum and the number of element in a RDD using aggregate

In [0]:
seqOp = (lambda result, e: (result[0] + e, result[1] + 1))
#for each element , we calculate the sum, and the count


combOp = (lambda result, e: (result[0] + e[0], result[1] + e[1]))
#Take the results and sum them

rdd.aggregate((0, 0), seqOp, combOp)


Out[65]: (10, 5)

Fonctionnement de la fonction du dessus

si on a un rdd qui est trois partitions : 0 1 2 3 4 | 5 6 7 8 9 | 10 11 12 13 14
on veut aggreate la somme et le nombre d'element

le résultat est en deux partie [somme, count]

au début résult[0,0]
puis on fait [0+0,0+1] (qui correspond à result[0] + e, result[1] + 1) ( somme et nombre d'element)
puis  [0+1,1+1] 
puis  [1+2,2+1] 
...

A la fin on a trois résult [10,5] [35,5], [60,5]

et on applique le second opérateur
[10+35,5+5]
puis [45+60,10+5]

ce qui finit par [105,15]

#### Calculate the sum, the count and the sum of square

#### Write a python program for spark that displays how many words in a script start by "a" and by "b" (lower and upper)

In [0]:
s = sc.parallelize("Bonjour je suis Albert et j'aime les abrico et les Bon bon")

In [0]:
s.flatMap(lambda e: e.split()).filter(lambda e:e.lower().startswith('a')).count()

Out[93]: 3

####Cwrite a wrord counting using pyspark, then display the words with their frequency

In [0]:
s = ["bnjour bonjour bonjour"]

In [0]:
rdd= sc.parallelize(s)
rdd.collect()

Out[97]: ['bnjour bonjour bonjour']

In [0]:
rdd.flatMap(lambda e: e.lower().split()) \
    .map(lambda e:(e,1)) \
    .reduceByKey(lambda r,v:r+v)\
    .sortBy(keyfunc=(lambda kv : (kv[1],kv[0])),ascending=False)\
    .collect()

Out[108]: [('bonjour', 2), ('bnjour', 1)]

# TP REVENUE

In [0]:
revenue_rdd = sc.wholeTextFiles('/FileStore/tables/TP1_revenue')

In [0]:
finalrdd = revenue_rdd.map(lambda kv :(kv[0].split('/')[-1].split('.txt')[0],kv[1]))\
               .flatMapValues(lambda v : v.split("\r\n"))\
                .map(lambda kv : (kv[0].split('_')[0],kv[0], kv[1].split()[0], kv[1].split()[1]))
finalrdd.collect()

Out[3]: [('anger', 'anger', 'JAN', '13'),
 ('anger', 'anger', 'FEB', '12'),
 ('anger', 'anger', 'MAR', '14'),
 ('anger', 'anger', 'APR', '15'),
 ('anger', 'anger', 'MAY', '12'),
 ('anger', 'anger', 'JUN', '15'),
 ('anger', 'anger', 'JUL', '19'),
 ('anger', 'anger', 'AUG', '15'),
 ('anger', 'anger', 'SEP', '13'),
 ('anger', 'anger', 'OCT', '8'),
 ('anger', 'anger', 'NOV', '14'),
 ('anger', 'anger', 'DEC', '16'),
 ('lyon', 'lyon', 'JAN', '13'),
 ('lyon', 'lyon', 'FEB', '12'),
 ('lyon', 'lyon', 'MAR', '14'),
 ('lyon', 'lyon', 'APR', '15'),
 ('lyon', 'lyon', 'MAY', '12'),
 ('lyon', 'lyon', 'JUN', '15'),
 ('lyon', 'lyon', 'JUL', '19'),
 ('lyon', 'lyon', 'AUG', '25'),
 ('lyon', 'lyon', 'SEP', '13'),
 ('lyon', 'lyon', 'OCT', '11'),
 ('lyon', 'lyon', 'NOV', '22'),
 ('lyon', 'lyon', 'DEC', '22'),
 ('marseilles', 'marseilles_1', 'JAN', '21'),
 ('marseilles', 'marseilles_1', 'FEB', '21'),
 ('marseilles', 'marseilles_1', 'MAR', '21'),
 ('marseilles', 'marseilles_1', 'APR', '27'),
 ('marseilles', '

In [0]:
df = spark.createDataFrame(finalrdd,schema=["city","store","month","revenue"])

In [0]:
df.show()

+-----+-----+-----+-------+
| city|store|month|revenue|
+-----+-----+-----+-------+
|anger|anger|  JAN|     13|
|anger|anger|  FEB|     12|
|anger|anger|  MAR|     14|
|anger|anger|  APR|     15|
|anger|anger|  MAY|     12|
|anger|anger|  JUN|     15|
|anger|anger|  JUL|     19|
|anger|anger|  AUG|     15|
|anger|anger|  SEP|     13|
|anger|anger|  OCT|      8|
|anger|anger|  NOV|     14|
|anger|anger|  DEC|     16|
| lyon| lyon|  JAN|     13|
| lyon| lyon|  FEB|     12|
| lyon| lyon|  MAR|     14|
| lyon| lyon|  APR|     15|
| lyon| lyon|  MAY|     12|
| lyon| lyon|  JUN|     15|
| lyon| lyon|  JUL|     19|
| lyon| lyon|  AUG|     25|
+-----+-----+-----+-------+
only showing top 20 rows



####Average montly income of all shops in france

In [0]:

print(finalrdd.map(lambda x: int(x[-1])).sum()/12)


301.5833333333333


####average monthly income of the shop in each city

In [0]:
#moyen plus opti
finalrdd.map(lambda x: (x[1],int(x[-1]))).reduceByKey(lambda s,e :s+e/12).collect()

Out[9]: [('anger', 25.749999999999996),
 ('lyon', 27.999999999999993),
 ('marseilles_1', 42.916666666666664),
 ('nice', 31.58333333333334),
 ('paris_2', 92.91666666666664),
 ('paris_3', 46.74999999999999),
 ('troyes', 37.083333333333336),
 ('marseilles_2', 29.333333333333332),
 ('nantes', 31.916666666666668),
 ('orlean', 28.249999999999996),
 ('paris_1', 96.41666666666666),
 ('rennes', 32.41666666666667),
 ('toulouse', 25.75)]

In [0]:
finalrdd.map(lambda x: (x[0],int(x[-1]))).groupByKey().mapValues(lambda x: round(sum(x),2))).collect()

Out[25]: [('anger', 166),
 ('lyon', 193),
 ('nice', 203),
 ('paris', 1568),
 ('troyes', 214),
 ('marseilles', 515),
 ('nantes', 207),
 ('orlean', 196),
 ('rennes', 180),
 ('toulouse', 177)]

####total revenue per city per year

In [0]:
finalrdd.map(lambda x: (x[0],int(x[-1]))).reduceByKey(lambda s,e :s+e).collect()

Out[33]: [('anger', 166),
 ('lyon', 193),
 ('nice', 203),
 ('paris', 1568),
 ('troyes', 214),
 ('marseilles', 515),
 ('nantes', 207),
 ('orlean', 196),
 ('rennes', 180),
 ('toulouse', 177)]

####total revenue per store per year

In [0]:
finalrdd.map(lambda x: (x[1],int(x[-1]))).reduceByKey(lambda s,e :s+e).collect()

Out[31]: [('anger', 166),
 ('lyon', 193),
 ('marseilles_1', 284),
 ('nice', 203),
 ('paris_2', 642),
 ('paris_3', 330),
 ('troyes', 214),
 ('marseilles_2', 231),
 ('nantes', 207),
 ('orlean', 196),
 ('paris_1', 596),
 ('rennes', 180),
 ('toulouse', 177)]

####The store with the best performance in each month

In [0]:
#Not optimal
finalrdd.map(lambda x:(x[2],(x[1],int(x[-1])))).groupByKey().mapValues(lambda l:max(l,key=lambda y:y[1])).collect()

#l is a list, so we take the max base of the city (the 1rst element)

Out[37]: [('APR', ('paris_1', 57)),
 ('MAY', ('paris_2', 72)),
 ('AUG', ('paris_2', 45)),
 ('JAN', ('paris_1', 51)),
 ('FEB', ('paris_2', 42)),
 ('MAR', ('paris_2', 44)),
 ('JUN', ('paris_2', 85)),
 ('JUL', ('paris_1', 61)),
 ('SEP', ('paris_2', 63)),
 ('OCT', ('paris_1', 68)),
 ('NOV', ('paris_2', 64)),
 ('DEC', ('paris_1', 71))]